In [ ]:
%matplotlib widget
import numpy as np
import random
import time
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="vpython")

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from vpython import *

In [ ]:
# Parameters
g = 9.81   # gravity
m = 5.     # mass

# **1. Modeling**

1. Write down a function in Python called `dynamics()` that takes as input the state $x \in \mathbb{R}^{4 \times 1}$ and controls $u \in \mathbb{R}^{2 \times 1}$ and returns $\dot{x} \in \mathbb{R}^{4 \times 1}.$

In [ ]:
def dynamics(x, u):
    x, y, x_dot, y_dot = x[0,0], x[1,0], x[2,0], x[3,0]

    rx = x/np.sqrt(x**2 + y**2)
    ry = y/np.sqrt(x**2 + y**2)
    r = np.array([[rx], [ry]])
    
    x_ddot = (rx*u[0,0] - ry*u[1,0])/m
    y_ddot = (ry*u[0,0] + rx*u[1,0])/m - g
    f = np.array([[x_dot, y_dot, x_ddot, y_ddot]]).T

    return f

2. Create a simple visualization of the system (e.g. by using the matplotlib library); you are
free to use any library for the visualization.

In [ ]:
# Visualization of Hopper Robot
def visualization(x, y):
    global scene, pivot, mass, rod
    
    scene = canvas(resizable=False, userzoom=False, userspin=False)
    scene.title = "Hopper Robot Vizualization"
    scene.width = 1280
    scene.height = 960
    
    scene.background = color.white # background color
    scene.center = vector(0, 0, 0) # background center
    
    scene.range = 0.8
    scene.camera.pos = vector(0, 0, 2)   # camera's position
    scene.camera.axis = vector(0, 0, -2) # camera's axis
    
    # Ground
    ground = box(pos=vector(0, 0, 0), size=vector(1.5, 0.05, 0.7), color=color.gray(0.7))
    pivot = sphere(pos=vector(0, 0.05, 0), radius=0.02, color=color.black)
    
    # Hopper robot
    mass = sphere(pos=vector(x, y, 0), radius=0.05, color=color.red, retain=200)
    rod = cylinder(pos=pivot.pos, axis=mass.pos - pivot.pos, radius=0.005, color=color.blue)

def visualization_update(x, y):
    global mass, rod
    
    rod.pos = pivot.pos
    rod.axis = mass.pos - pivot.pos
    mass.pos = vector(x, y, 0)

3. Discretize the system using:

In [ ]:
# a. Euler integration
def euler(x, u):
    return x + dynamics(x,u) * dt

In [ ]:
# b. Semi-Implicit Euler integration
def semi_implicit_euler(x, u):
    f = dynamics(x,u)
    
    q = np.array([[x[0,0], x[1,0]]]).T
    v = np.array([[x[2,0], x[3,0]]]).T
    q_dot = np.array([[f[0,0], f[1,0]]]).T
    v_dot = np.array([[f[2,0], f[3,0]]]).T

    v_n = v + v_dot * dt
    q_n = q + v_n * dt
    
    return np.block([[q_n, v_n]]).T

In [ ]:
# c. Runge-Kutta 4th Order integration
def rk4(x, u):
    f1 = dynamics(x,u)
    f2 = dynamics(x + f1 * dt/2,u)
    f3 = dynamics(x + f2 * dt/2,u)
    f4 = dynamics(x + f3 * dt,u)
    
    return x + dt/6 * (f1 + 2*f2 + 2*f3 + f4)

In [ ]:
# d. Midpoint integration
def midpoint(x, u):
    x_m = x + dynamics(x,u) * dt/2
    
    return x + dynamics(x_m,u) * dt

4. Create a simulation loop that works with any of the above integrators; **the initial state of
the system should be** $x_0 = [0\,\,\,1\,\,\,0\,\,\,0]^T$

6. Write a simple PD-controller for the position of the body (e.g. to stay in place) and use it with the simulation loop.

In [ ]:
def pd_controller(desired_x, x, kp=90., kd=40.):
    pos_error = desired_x[:2,0] - x[:2,0]
    vel_error = desired_x[2:,0] - x[2:,0]

    F = (kp * pos_error + kd * vel_error).reshape(2,1)
    Fx = F[0,0]
    Fy = F[1,0] + m * g

    theta = np.arctan2(x[0,0],x[1,0])
    f = Fx * np.sin(theta) + Fy * np.cos(theta)
    tau = -Fx * np.cos(theta) + Fy * np.sin(theta)

    return np.array([[f, tau]]).T

In [ ]:
x = np.array([[0., 1., 0., 0.]]).T # initial state
u = np.array([[0., 0.]]).T         # initial control input
    
visualization(x[0,0], x[1,0])

xs = []
t = 26.5            # total simulation time
dt = 0.15           # time step
N = round(t/dt) + 1 # number of steps

for i in range(N):
    rate(30)
    
    if (x[1,0] < 0.12):
        x[1,0] = 0.12

    # Sinusoidal motion on the vertical axis: [x, y, x_dot, y_dot]
    desired_x = np.array([[                  0.5],
                          [0.4 + 0.2 * np.sin(t)],
                          [                   0.],
                          [      0.2 * np.cos(t)]])

    # Control input: [f, τ]
    u = pd_controller(desired_x, x)

    # State update
    xs.append(np.copy(x))
    visualization_update(x[0,0], x[1,0])

    # Integrators
    # x = euler(x, u)
    # x = semi_implicit_euler(x, u)
    x = rk4(x, u)
    # x = midpoint(x, u)

    t += dt
    
# Let's plot it!
xs = np.array(xs)

fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(121)

ax1.plot([n*dt for n in range(len(xs))], xs[:,0], label=r'x')
ax1.plot([n*dt for n in range(len(xs))], xs[:,1], label=r'y')
ax1.legend()

ax2 = fig.add_subplot(122)

ax2.plot([n*dt for n in range(len(xs))], xs[:,2], label=r'$\dot{x}$')
ax2.plot([n*dt for n in range(len(xs))], xs[:,3], label=r'$\dot{y}$')
ax2.legend()
plt.show()